# Import GPT Wrapper

In [ ]:
import gpt_wrapper

gpt_wrapper.api_base = "http://mnlp-backend-938795011.eu-central-1.elb.amazonaws.com"
gpt_wrapper.api_key = "XXX"

In [3]:
from gpt_wrapper.chat import Chat

chat1 = Chat.create("Test Chat")
message = chat1.ask(content="Who won the FIFA championship in 2018?")
print(message)

The French national team won the FIFA World Cup in 2018.


In [4]:
Chat.budget()

{'limit': 10000000, 'usage': 1310416}

# Load data

In [ ]:
import pandas as pd 

data = pd.read_json("data/330072.json")

display(data)

,course_id,question_id,question_body,question_options
0,15000,66,How many elements of the input contribute to a...,None
1,15006,4170,"Given the following classes:\n• class Pair[+U,...",None
2,15003,7361,"A sequential object is a tuple T = (Q, q0, O, ...",None
3,15000,67,Consider an operation we will call scanRight1 ...,None
4,0,6154,Which of the following is well preserved by 2G?,"[Confidentiality, Message Integrity, Challenge..."
...,...,...,...,...
95,15000,32,Is calling <code>parGroupyBy3</code> determini...,None
96,15125,3658,Implement weigthing estimation of kNN classifi...,None
97,15041,2744,What is the mean squared error of $f$ for a sa...,"[ $||y - f(\textbf{x},W)||^2 $ , $||y - f(\te..."
98,5,6698,\t Consider a source $S$ with some distributio...,"[True, False]"


In [72]:
columns = ["course_id", "question_id", "question", "A_chat_id", "B_chat_id", "A", "B", "ranking_criteria"]
output_df = pd.DataFrame(columns=columns)

# group by the 'course_id' column and iterate over the groups
for course_id, group in data.groupby("course_id"):
    chat_1 = Chat.create("chat_1")
    chat_2 = Chat.create("chat_2")
    for i, row in group.iterrows():
        question = row["question_body"]

        if row["question_options"]:
            question += "\nChoose among the following options: \n" + ", \n".join(row["question_options"])

        question_1 = question + "\nExplain your reasoning step by step. " 
        question_2 = question + "\nJustify your answer. "
        
        answer_1 = chat_1.ask(content=question_1)
        answer_2 = chat_2.ask(content=question_2)

        output = {
            "course_id": course_id,
            "question_id": row["question_id"],
            "question": row["question_body"],
            "A_chat_id": answer_1.to_dict()["chat_id"],
            "B_chat_id": answer_2.to_dict()["chat_id"],
            "A": answer_1.to_dict()["content"],
            "B": answer_2.to_dict()["content"],
            "ranking_criteria": {
                        "overall": "",
                        "correctness": "",
                        "relevance": "",
                        "clarity": "",
                        "completeness": "",
                        "other": "Conciseness: ; Engagement: "
            }
        }
        output_df = pd.concat([output_df, pd.DataFrame([output], columns=columns)], ignore_index=True)
        
output_df.to_json("data/330072.json", orient="records")
        
    